In this tutorial we will be solving the classical MNIST problem and deploy our solution on an android device.

## Before we go ahead lets see what we are going to do

- Download the data 
- Define learning rate and training step
- create the model
- define performance metrices
- run the model
- analyzing change in accuracy and loss with iteration 

<br></br>

Import necessary packages

In [ ]:
import tensorflow as tf
print('TensorFlow version: ' + tf.__version__)

### Download the data
First of all we download the input data and investigate its nature. 

Running this for first time will take some time as it'll download the dataset and save in repo's root directory **MNIST_data**
- Input is a 28x28(=784) pixels image, stored as a liner float array

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data as mnist_data
mnist = mnist_data.read_data_sets("../MNIST_data", one_hot=True, validation_size=0)

x_train = mnist.train.images # we will not be using these to feed in data
y_train = mnist.train.labels # instead we will be using next_batch function to train in batches
x_test = mnist.test.images
y_test = mnist.test.labels

print ('We have '+str(x_train.shape[0])+' training examples in dataset')
print ('We have '+str(x_train.shape[1])+' feature points(basically pixels) in each input example')

In [ ]:
TUTORIAL_NAME = 'Tutorial2'
MODEL_NAME = 'mnistTFonAndroid'
SAVED_MODEL_PATH = '../' + TUTORIAL_NAME+'_Saved_model/'

### Learning rate and Training step

**OK. This section has a great deal of learning so those who are new to deep learning kindly read carefully**

Lets lay down how do Nerual nets work

- We create computation nodes with some weights associated to them. 
- These weights are used to create predictions from the input we give in
- If prediction is correct that's good, ELSE WE ADJUST THOSE WEIGHTS TO IMPROVE PREDICTION ACCURACY
- To aid in adjusting those weights we create some performance parameters like ACCURACY and LOSS


You remember when I introduced Parameters(weights) I said they have some magic values. That magic value is obtained by training our model. Training our model basically is 
- We initialize the Parameters with random value 
- We feed in our input and check if model gives correct output
- If no, we adjust the parameters on basis of how wrong the prediction was
- We repeat this for all the example several time, doing this repeatedly 

How many times we feed the whole dataset into model is **TRAIN_STEPS**

In this how much to adjust the weights is determined by **LEARNING_RATE**

Learning rate is the quantum of step you take in order to reach the optimal accuracy point

<img src="../images/t2_1_learning_rate.jpeg" alt="Drawing" style="width: 400px;"/>


(When you run this example below you'll observe a large learning rate is ok in starting but soon it becomes ineffective to increase accuracy beyond a certain point)

In [ ]:
LEARNING_RATE = 0.1
TRAIN_STEPS = 1000

### Create the model

Defining our network, a single node network with two input nodes

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, 784], name='modelInput')
Y_ = tf.placeholder(tf.float32, shape=[None, 10])
W = tf.Variable(tf.zeros([784,10]), name='modelWeights')
b = tf.Variable(tf.zeros([10]), name='modelBias')
Y = tf.nn.softmax(tf.matmul(X,W) + b, name='modelOutput')

### Metrices to learn and judge our model

See the training variable below , It takes in LEARNING_RATE and will adjust the weights on basis of cross_entropy we calculate. Cross_entropy is just how wrong our prediction was on a lograthmic scale.

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y_ * tf.log(Y), reduction_indices=[1]))
training = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Pay attention here

In last tutorial we manually converted a variable and deployed on android. Here we will use the freeze_graph api provided by google.

Before that
- During training our weights W and b will learn some value. We will periodically save those learned parameters onto harddisk
- Learned variables are saved as checkpoint file **.ckpt** using the tf.train.Saver() API.
- code below will be saving check point files for you
```python
    if i%500 == 0:
        out = saver.save(sess, SAVED_MODEL_PATH + MODEL_NAME + '.ckpt', global_step=i)
```

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()

sess.run(init)
saver = tf.train.Saver()

### Training our model
Below is the code where our model is trained on the input data. Once you run it you will see that accuracy increases till a certain point (roughly around 500 steps) after which we did  not see any improvements.

- our maximum accuracy is where blue line meets the curve beyond which it bounces back 
- to go into further into purple region in need to decrease the learning rate

<img src="../images/t2_2_lr_explain.jpeg" alt="Drawing" style="width: 500px;"/>



In [ ]:
for i in range(TRAIN_STEPS+1):
    sess.run(training, feed_dict={X: x_train, Y_: y_train})
    if i%100 == 0:
        print('Training Step:' + str(i) + 
              '  Accuracy =  ' + str(sess.run(accuracy, feed_dict={X: x_test, Y_: y_test})) + 
              '  Loss = ' + str(sess.run(cross_entropy, {X: x_train, Y_: y_train}))
             )
    if i%500 == 0:
        out = saver.save(sess, SAVED_MODEL_PATH + MODEL_NAME + '.ckpt', global_step=i)

Lets save our graph defination as well.

In [ ]:
tf.train.write_graph(sess.graph_def, SAVED_MODEL_PATH , MODEL_NAME + '.pbtxt')
tf.train.write_graph(sess.graph_def, SAVED_MODEL_PATH , MODEL_NAME + '.pb',as_text=False)

So now we have our model defination in a .pb file and values of variables learned in ckpt file.
Our next task is to merge the .pb file and .ckpt file into a single .pb file and freeze all the variable nodes into constant nodes.

Remember the input checkpoint file shoud be the latest one saved, kindly go and check the Tutorial2_Saved_model folder.

In [ ]:
from tensorflow.python.tools import freeze_graph

# Freeze the graph
input_graph = SAVED_MODEL_PATH+MODEL_NAME+'.pb'
input_saver = ""
input_binary = True
input_checkpoint = SAVED_MODEL_PATH+MODEL_NAME+'.ckpt-'+str(TRAIN_STEPS)
output_node_names = 'modelOutput'
restore_op_name = 'save/restore_all'
filename_tensor_name = 'save/Const:0'
output_graph = SAVED_MODEL_PATH+'frozen_'+MODEL_NAME+'.pb'
clear_devices = True
initializer_nodes = ""
variable_names_blacklist = ""

freeze_graph.freeze_graph(
    input_graph,
    input_saver,
    input_binary,
    input_checkpoint,
    output_node_names,
    restore_op_name,
    filename_tensor_name,
    output_graph,
    clear_devices,
    initializer_nodes,
    variable_names_blacklist
)

After you have executed all the code above you will find 
- **mnistTFonAndroid.pbtxt** file, open it see your graph defination there
- **frozen_mnistTFonAndroid.pb** file , which we have to copy to our assets folder in android app, change the name by deleting the underscore in _frozen_mnistTFonAndroid.pb in **.MNISTActivity.java** and run the app.

Complete code:

In [ ]:
import tensorflow as tf
print('TensorFlow version: ' + tf.__version__)


from tensorflow.examples.tutorials.mnist import input_data as mnist_data
mnist = mnist_data.read_data_sets("../MNIST_data", one_hot=True, validation_size=0)

x_train = mnist.train.images # we will not be using these to feed in data
y_train = mnist.train.labels # instead we will be using next_batch function to train in batches
x_test = mnist.test.images
y_test = mnist.test.labels

print ('We have '+str(x_train.shape[0])+' training examples in dataset')
print ('We have '+str(x_train.shape[1])+' feature points(basically pixels) in each input example')


TUTORIAL_NAME = 'Tutorial2'
MODEL_NAME = 'mnistTFonAndroid'
SAVED_MODEL_PATH = '../' + TUTORIAL_NAME+'_Saved_model/'


LEARNING_RATE = 0.1
TRAIN_STEPS = 2000


X = tf.placeholder(tf.float32, shape=[None, 784], name='modelInput')
Y_ = tf.placeholder(tf.float32, shape=[None, 10])
W = tf.Variable(tf.zeros([784,10]), name='modelWeights')
b = tf.Variable(tf.zeros([10]), name='modelBias')
Y = tf.nn.softmax(tf.matmul(X,W) + b, name='modelOutput')


cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y_ * tf.log(Y), reduction_indices=[1]))
training = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


sess = tf.Session()
init = tf.global_variables_initializer()

sess.run(init)
saver = tf.train.Saver()


for i in range(TRAIN_STEPS+1):
    sess.run(training, feed_dict={X: x_train, Y_: y_train})
    if i%100 == 0:
        print('Training Step:' + str(i) + 
              '  Accuracy =  ' + str(sess.run(accuracy, feed_dict={X: x_test, Y_: y_test})) + 
              '  Loss = ' + str(sess.run(cross_entropy, {X: x_train, Y_: y_train}))
             )
    if i%500 == 0:
        out = saver.save(sess, SAVED_MODEL_PATH + MODEL_NAME + '.ckpt', global_step=i)
           
        
tf.train.write_graph(sess.graph_def, SAVED_MODEL_PATH , MODEL_NAME + '.pbtxt')
tf.train.write_graph(sess.graph_def, SAVED_MODEL_PATH , MODEL_NAME + '.pb',as_text=False)


from tensorflow.python.tools import freeze_graph

# Freeze the graph
input_graph = SAVED_MODEL_PATH+MODEL_NAME+'.pb'
input_saver = ""
input_binary = True
input_checkpoint = SAVED_MODEL_PATH+MODEL_NAME+'.ckpt-'+str(TRAIN_STEPS)
output_node_names = 'modelOutput'
restore_op_name = 'save/restore_all'
filename_tensor_name = 'save/Const:0'
output_graph = SAVED_MODEL_PATH+'frozen_'+MODEL_NAME+'.pb'
clear_devices = True
initializer_nodes = ""
variable_names_blacklist = ""

freeze_graph.freeze_graph(
    input_graph,
    input_saver,
    input_binary,
    input_checkpoint,
    output_node_names,
    restore_op_name,
    filename_tensor_name,
    output_graph,
    clear_devices,
    initializer_nodes,
    variable_names_blacklist
)